# 필요 모듈 임포트

In [39]:
import re
import pandas as pd
import numpy as np
from konlpy.tag import Kkma
import re
import unicodedata
import re
import nltk
from string import whitespace, punctuation
from nltk.tokenize import sent_tokenize
import warnings

warnings.filterwarnings('ignore')

In [40]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/parkjunhyeong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 데이터 로드

In [114]:
#기간별 뉴스 데이터 로드 및 병합
load_path = './'
news_df1 = pd.read_excel(path + 'Analysis_NewsData_20190501_20210930_(청년정책)&&~(rt__독취사)_BY_ARGOS.xlsx', sheet_name = '뉴스', skiprows = 1)
news_df2 = pd.read_excel(path + 'Analysis_NewsData_20211001_20230626_(청년정책)&&~(rt__독취사)_BY_ARGOS.xlsx', sheet_name = '뉴스', skiprows = 1)
news_df = pd.concat([news_df1, news_df2], ignore_index = True)

In [115]:
news_df

,작성일,작성자,제목,내용
0,2021/02/01,스포츠서울,"광양시, 2021년 청년정책 시행계획 수립 추진",2021년 청년정책 시행계획.(제공=광양시) [광양=스포츠서울 이정진 기자] 전남 ...
1,2021/08/31,국민일보,기재부 등 4개 부처에 청년정책 전담부서 신설,교육부 등 5개 부처엔 청년정책 전담인력 보강…9개 부처 직제 개정령안 국무회의 의...
2,2021/02/01,국민일보,"광양시, 청년정책 추진 박차··‘청년 꿈 이루는 도시’",전남 광양시청 전경<사진=광양시 제공> 전남 광양시가 올해 청년정책 시행계획을 수립...
3,2019/11/21,경향신문,“일자리가 다가 아냐” 청년의 삶 이해해야 청년정책도 변한다 [청년 없는 청년정치],"ㆍ③ 알맹이 없는 청년정책 일러스트 | 김상민 기자 20대 국회, 청년법안 처리율 ..."
4,2019/11/15,프레시안,"청년 정책, 수당이 전부가 아니다",[내가 만드는 복지국가] 시대 변화 인정하고 불공정·불평등 물어야… [기현주 서울...
...,...,...,...,...
18922,2021/10/01,뉴스1,김돈곤 청양군수 “지방소멸 파도 극복하려면 혼신 다해 노 저어야”,청양군민의 날 기념사에서 역량 결집 당부 김돈곤 군수가 1일 청양문예회관 대공연장에...
18923,2021/10/01,헤럴드경제,"관악구, 청년 소상공인이 만든 도시락으로 취약층 지원",‘행복나눔 도시락’ 만들 청년 소상공인 10월 초까지 모집 관악구 청사 전경. [관...
18924,2021/10/01,아시아경제,관악구 ‘청년 소상공인 행복나눔 도시락 지원 사업' 추진,"[서울시 자치구 뉴스]관악구, 청년 소상공인 긴급 판로 제공 및 복지 사각지대 안전..."
18925,2021/10/01,이데일리,[기고]청년들과 함께 만드는 정책,성장현 서울 용산구청장. [성장현 용산구청장] “청년주택에서 잘 지내고 있습니다. ...


# 전처리

In [116]:
# 140자 이하의 기사 제외
news_df = news_df[news_df['내용'].str.len() > 140]
news_df.reset_index(inplace = True, drop = True)
news_df.shape

(18637, 4)

In [117]:
# 기사 제목 및 본문 공백 처리 
# 문장 단위 semantic을 기준으로 임베딩할 것이므로 \n \t 등을 모두 공백으로 처리
pattern_whitespace = re.compile(f'[{whitespace}]+')

# 기사 본문 공백 처리
news_df['내용_n'] = news_df['내용'].str.replace(pattern_whitespace, ' ')
news_df['내용_n'] = news_df['내용_n'].apply(lambda x : unicodedata.normalize('NFC', x)).str.strip()
# 기사 제목 공백 처리 
news_df['제목_n'] = news_df['제목'].str.replace(pattern_whitespace, ' ')
news_df['제목_n'] = news_df['제목_n'].apply(lambda x : unicodedata.normalize('NFC', x)).str.strip()

In [118]:
# 전처리 함수 정의

# 이메일, url, byline, 기타 특수 문구 제거 함수
def clean_byline(text):
    # 이메일
    pattern_email = re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE)
    # url
    pattern_url = re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+(?:\.[-_0-9a-z]+)+', flags=re.IGNORECASE)
    # 마지막 문장 바이라인
    pattern_byline1 = re.compile(r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|글©|©|ⓒ|\@|\/|=|▶|>|<|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$')
    # 본문 시작 전 바이라인 제거
    pattern_byline2 = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)')
    # TBD : 기타 제거 문구들 고도화 여지 있음
    # 기타
    pattern_others1 = re.compile(r'\[[^\]]*기자[^\]]*\]')
    pattern_others2 = re.compile(r'\b\w+\s+기자\s+=\s+')
    pattern_others3 = re.compile(r'\([^)]*=[^)]*\)')
    pattern_others4 = re.compile(r'【[^)]*=[^)]*】')
    
    result = pattern_email.sub('', text)
    result = pattern_url.sub('', result)
    result = pattern_byline1.sub('.', result)
    result = pattern_byline2.sub('', result)
    result = pattern_others1.sub('', result)
    result = pattern_others2.sub('', result)
    result = pattern_others3.sub('', result)
    result = pattern_others4.sub('', result).strip()

    return result

# %, 숫자, 알파벳, 한글만 남기고 나머지 제거하는 함수 (성능 비교를 위해)
def text_filter(text):

    punct_except_percent = ''.join([chr for chr in punctuation if chr != '%'])
    whitespace_convert_pattern = re.compile(f'[{whitespace}{punct_except_percent}]+')
    exclude_pattern = re.compile(r'[^\% 0-9a-zA-Zㄱ-ㅣ가-힣]+')

    result = whitespace_convert_pattern.sub(' ', text)
    result = exclude_pattern.sub(' ', result).strip()
    result = whitespace_convert_pattern.sub(' ', result)
    
    return result

In [119]:
#전처리 ver1는 1. clean_byline, 2. 문장별 분리
#전처리 ver2는 2. clean_byline, 2. 문장별 분리, 3. text_filter

# ver1
news_df['내용_n1'] = news_df['내용_n'].apply(clean_byline)
news_df['내용_n1'] = news_df['내용_n1'].apply(sent_tokenize)

# ver2
news_df['내용_n2'] = news_df['내용_n'].apply(clean_byline)
news_df['내용_n2'] = news_df['내용_n2'].apply(sent_tokenize)
news_df['내용_n2'] = news_df['내용_n2'].apply(lambda sentences: [text_filter(sentence) for sentence in sentences
                                                            if len(text_filter(sentence)) > 0])

In [120]:
#전처리 결과 확인
idx = 1321
print(news_df.loc[idx,'내용'], '\n\n')
print(news_df.loc[idx,'내용_n1'], '\n\n')
print(news_df.loc[idx,'내용_n2'], '\n\n')

【파이낸셜뉴스 울산=최수상 기자】 청년의 지역사회 참여를 유도하고 청년정책 수요자가 청년정책 결정 과정에 직접 참여할 수 있는 ‘울산시 청년정책위원회’가 구성됐다. 18일 울산시에 따르면 울산시 청년정책위원회는 당연직으로 시장 및 청년정책 유관 국장 9명, 분야별(정책 기반 조성, 일자리, 창업, 문화·청년활동, 주거복지) 전문위원 5명, 청년위원 6명 등 총 20명으로 구성됐다. 울산시 시정 10대 핵심과제에 올라있는 ‘청년 기 살리기’ 와 관련된 청년정책 컨트롤타워인 셈이다.       당연직을 제외하면 모두 35세 이하로 젊은 위원으로 구성한 것도 같은 맥락이다. 청년위원은 청년(만 19세~34세) 대상 공개 모집 및 주제발표 등 소정의 절차를 거쳐 선발했다.     분야별 전문위원 또한 전문성과 소통성,청년의 시선을 담아 낼 수 있는 젊은 층으로 구성됐다. 앞으로 청년정책위원회는 울산시 청년정책 사업의 주요 사항을 심의, 조정하게 된다. 지난 3월 초 위원회 구성을 완료할 계획이었으나 코로나19로 연기됨에 따라 위원회 구성 또한 일정이 연기된 바 있다. 울산시 관계자는 “청년이 참여하고 청년이 만들어 가는 청년정책들이 잘 운영될 수 있도록 적극 지원하겠다.”라고 말했다.           ulsan@fnnews.com 최수상 기자             ※ 


['청년의 지역사회 참여를 유도하고 청년정책 수요자가 청년정책 결정 과정에 직접 참여할 수 있는 ‘울산시 청년정책위원회’가 구성됐다.', '18일 울산시에 따르면 울산시 청년정책위원회는 당연직으로 시장 및 청년정책 유관 국장 9명, 분야별(정책 기반 조성, 일자리, 창업, 문화·청년활동, 주거복지) 전문위원 5명, 청년위원 6명 등 총 20명으로 구성됐다.', '울산시 시정 10대 핵심과제에 올라있는 ‘청년 기 살리기’ 와 관련된 청년정책 컨트롤타워인 셈이다.', '당연직을 제외하면 모두 35세 이하로 젊은 위원으로 구성한 것도 같은 맥락이다.', '청년위원은 청년(만 19세~34세) 대상 공개 

# 전처리 결과 저장

In [122]:
store_path = './'
news_df.to_json(store_path + '청년정책_preprocessed_v0.1.json')